<a href="https://colab.research.google.com/github/futurekasuga/100knock_pyspark/blob/main/preprocess_knock_Python_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - Python
# for Google Colab

# Sparkの準備
## Spark参考ドキュメント
- https://spark.apache.org/docs/3.2.0/api/python/reference/index.html

In [12]:
!pip install --upgrade pip
!pip install pyspark
!pip install -U pandas numpy scikit-learn imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 2.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=756f97e707c32f3765ca01dfdd1f559b0868ad42db1be09b28016b1247447b91
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━

In [13]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("IAB") \
    .getOrCreate()

In [14]:
spark

# 処理対象のデータをgitで取得

In [ ]:
!git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
cd 100knocks-preprocess/docker/work/data/

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません


In [15]:
# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

# Spark SQL用
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Spark DataFrame用
import pyspark.pandas as ps

In [16]:
# データを読み込み
# inferSchemaがfalseの場合、すべてのカラムはstringとなる。(datetime不可。timestamp可)
df_customer = spark.read\
              .format("csv")\
              .options(header="true", inferSchema="true")\
              .load('./customer.csv')
df_category = spark.read\
              .format("csv")\
              .options(header="true", inferSchema="true")\
              .load('./category.csv')
df_product = spark.read\
              .format("csv")\
              .options(header="true", inferSchema="true")\
              .load('./product.csv')
df_receipt = spark.read\
              .format("csv")\
              .options(header="true", inferSchema="true")\
              .load('./receipt.csv')
df_store = spark.read\
            .format("csv")\
            .options(header="true", inferSchema="true")\
            .load('./store.csv')
df_geocode = spark.read\
            .format("csv")\
            .options(header="true", inferSchema="true")\
            .load('./geocode.csv')

In [ ]:
# 業務環境では実行不可
# tmp_test = ps.read_csv('./geocode.csv')
# tmp_test.head()

/usr/local/lib/python3.7/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
0,060-0000,北海道,札幌市中央区,None,None,None,北海道札幌市中央区,141.34103,43.05513
1,064-0941,北海道,札幌市中央区,旭ケ丘,None,None,北海道札幌市中央区旭ケ丘,141.31972,43.04223
2,060-0042,北海道,札幌市中央区,大通西,None,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102
3,060-0042,北海道,札幌市中央区,大通西,None,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080
4,060-0042,北海道,札幌市中央区,大通西,None,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|
| 20181205| 1543968000|  S14024|      1102|             2|CS024514000042|P080101005|       

---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [ ]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "amount"]).show(10)

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20181103|CS006214000001|P070305012|   158|
| 20181118|CS008415000097|P070701017|    81|
| 20170712|CS028414000014|P060101005|   170|
| 20190205|ZZ000000000000|P050301001|    25|
| 20180821|CS025415000050|P060102007|    90|
| 20190605|CS003515000195|P050102002|   138|
| 20181205|CS024514000042|P080101005|    30|
| 20190922|CS040415000178|P070501004|   128|
| 20170504|ZZ000000000000|P071302010|   770|
| 20191010|CS027514000015|P071101003|   680|
+---------+--------------+----------+------+
only showing top 10 rows



---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "amount"])\
          .withColumnRenamed("sales_ymd", "sales_date").show(10)

+----------+--------------+----------+------+
|sales_date|   customer_id|product_cd|amount|
+----------+--------------+----------+------+
|  20181103|CS006214000001|P070305012|   158|
|  20181118|CS008415000097|P070701017|    81|
|  20170712|CS028414000014|P060101005|   170|
|  20190205|ZZ000000000000|P050301001|    25|
|  20180821|CS025415000050|P060102007|    90|
|  20190605|CS003515000195|P050102002|   138|
|  20181205|CS024514000042|P080101005|    30|
|  20190922|CS040415000178|P070501004|   128|
|  20170504|ZZ000000000000|P071302010|   770|
|  20191010|CS027514000015|P071101003|   680|
+----------+--------------+----------+------+
only showing top 10 rows



---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "amount"])\
          .filter(df_receipt["customer_id"]=="CS018205000001").show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "amount"])\
          .filter((df_receipt["customer_id"]=="CS018205000001") & (df_receipt["amount"]>=1000)).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "quantity", "amount"])\
          .filter((df_receipt["customer_id"]=="CS018205000001") & ((df_receipt["amount"]>=1000) | (df_receipt["quantity"]>5))).show()

+---------+--------------+----------+--------+------+
|sales_ymd|   customer_id|product_cd|quantity|amount|
+---------+--------------+----------+--------+------+
| 20180911|CS018205000001|P071401012|       1|  2200|
| 20180414|CS018205000001|P060104007|       6|   600|
| 20190226|CS018205000001|P071401020|       1|  2200|
| 20180911|CS018205000001|P071401005|       1|  1100|
+---------+--------------+----------+--------+------+



---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "amount"])\
          .filter((F.col("customer_id")=="CS018205000001") & ((F.col("amount")>=1000) & (F.col("amount")<=2000))).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_receipt.select(["sales_ymd", "customer_id", "product_cd", "amount"])\
          .filter((F.col("customer_id")=="CS018205000001") & (F.col("product_cd")!="P071401019")).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



In [20]:
# AI Programmerのテスト
# pysparkでレシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。顧客ID（customer_id）が"CS018205000001"商品コード（product_cd）が"P071401019"以外である
# df_receipt.select("sales_ymd","customer_id","product_cd","amount").where(col("customer_id") != "CS018205000001" and col("product_cd") != "P071401019")

df_receipt.select("sales_ymd","customer_id","product_cd","amount").where((F.col("customer_id") != "CS018205000001") & (F.col("product_cd") != "P071401019")).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20181103|CS006214000001|P070305012|   158|
| 20181118|CS008415000097|P070701017|    81|
| 20170712|CS028414000014|P060101005|   170|
| 20190205|ZZ000000000000|P050301001|    25|
| 20180821|CS025415000050|P060102007|    90|
| 20190605|CS003515000195|P050102002|   138|
| 20181205|CS024514000042|P080101005|    30|
| 20190922|CS040415000178|P070501004|   128|
| 20170504|ZZ000000000000|P071302010|   770|
| 20191010|CS027514000015|P071101003|   680|
| 20190918|CS025415000134|P070401002|   138|
| 20171010|CS021515000126|P040402001|   228|
| 20180506|CS039414000052|P059001019|   428|
| 20190326|CS016215000032|P091401190|   780|
| 20180329|ZZ000000000000|P050104001|   115|
| 20170116|ZZ000000000000|P080803001|   100|
| 20190621|ZZ000000000000|P040102001|   268|
| 20181225|ZZ000000000000|P071401002|  1100|
| 20190603|CS026515000042|P070504016|   188|
| 20190606

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.filter((F.col("prefecture_cd")!="13") & (F.col("floor_area")<=900)).show()

+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                           address|                          address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|  S14046|  北山田店|           14|  神奈川県|  神奈川県横浜市都筑区北山田一丁目| カナガワケンヨコハマシツヅキクキタ...|045-123-4049| 139.5916|35.56189|     831.0|
|  S14011|日吉本町店|           14|  神奈川県|神奈川県横浜市港北区日吉本町四丁目| カナガワケンヨコハマシコウホククヒ...|045-123-4033| 139.6316|35.54655|     890.0|
|  S12013|  習志野店|           12|    千葉県|          千葉県習志野市芝園一丁目|チバケンナラシノシシバゾノイッチョウメ|047-123-4002|  140.022|35.66122|     808.0|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+-----

---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_store.filter(F.col("store_cd").rlike('S14.*')).show(10)

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14036|相模原中央店|           14|  神奈川県|            神奈川県相模原市中央二丁目|   カナガワケンサガミハラシチュウオウ...|042-123-4045| 139.3716|35.57327|    1679.0|
|  S14040|    長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-1

---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.filter(F.col("customer_id").rlike('.*1$')).show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01| 66| 136-0076|       東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
|CS028811000001|  堀井 かおり|        1|  女性|1933-03-27| 86| 245-0016| 神奈川県横浜市泉区和泉町*****...|              S14028|        20160115|0-00000000-0|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021|神奈川県横浜市緑区北八朔町****...|              S14040|        20151101|1-20091025-4|
|CS028314000011|  小菅 あおい|        1|  女性|1983-11-26| 35| 246-0038| 神奈川県横浜市瀬谷区宮沢*****...|              S14

---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
df_store.filter(F.col("address").rlike('.*横浜市.*')).show()

+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|    菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|  阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14040|  長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-123-4046| 139.4994|35.52398|    1548.0|
|  S14050|阿久和西店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4053| 139.4961

---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.filter(F.col("status_cd").rlike('^[A-F]')).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|      東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
|CS015414000103|    奥野 陽子|        1|  女性|1977-08-09| 41| 136-0073|        東京都江東区北砂**********|              S13015|        20150722|B-20100609-B|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS029415000023|    梅田 里穂|        1|  女性|1976-01-17| 43| 279-0043|      千葉県浦安市富士見**********|     

---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.filter(F.col("status_cd").rlike('[1-9]$')).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS001215000145|    田崎 美紀|        1|  女性|1995-03-29| 24| 144-0055|      東京都大田区仲六郷**********|              S13001|        20170605|6-20090929-2|
|CS033513000180|      安斎 遥|        1|  女性|1962-07-11| 56| 241-0823|  神奈川県横浜市旭区善部町*****...|              S14033|        20150728|6-20080506-5|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021| 神奈川県横浜市緑区北八朔町****...|          

---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.filter((F.col("status_cd").rlike('^[A-F].*[1-9]$'))).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS022513000105|  島村 貴美子|        1|  女性|1962-03-12| 57| 249-0002|    神奈川県逗子市山の根**********|              S14022|        20150320|A-20091115-7|
|CS001515000096|    水野 陽子|        9|  不明|1960-11-29| 58| 144-0053|    東京都大田区蒲田本町**********|              S13001|        20150614|A-20100724-7|
|CS013615000053|    西脇 季衣|        1|  女性|1953-10-18| 65| 261-0026|  千葉県千葉市美浜区幕張西*****...|          

---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df_store.filter(F.col("tel_no").rlike('[0-9]{3}-[0-9]{3}-[0-9]{4}')).show()

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S12014|    千草台店|           12|    千葉県|        千葉県千葉市稲毛区千草台一丁目|   チバケンチバシイナゲクチグサダイイ...|043-123-4003|  140.118|35.63559|    1698.0|
|  S13002|    国分寺店|           13|    東京都|              東京都国分寺市本多二丁目|トウキョウトコクブンジシホンダニチョウメ|042-123-4008| 139.4802|35.70566|    1735.0|
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...

---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.orderBy(F.col("birth_day")).show(10)

+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                         address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|CS003813000014|  村山 菜々美|        1|  女性|1928-11-26| 90| 182-0007|    東京都調布市菊野台**********|              S13003|        20160214|0-00000000-0|
|CS026813000004|    吉村 朝陽|        1|  女性|1928-12-14| 90| 251-0043| 神奈川県藤沢市辻堂元町******...|              S14026|        20150723|0-00000000-0|
|CS018811000003|    熊沢 美里|        1|  女性|1929-01-07| 90| 204-0004|      東京都清瀬市野塩**********|              S13018|        20150403|0-00000000-0|
|CS027803000004|    内村 拓郎|        0|  男性|1929-01-12| 90| 251-0031|神奈川県藤沢市鵠沼藤が谷*****...|              S140

---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.orderBy(F.col("birth_day").desc()).show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS035114000004|    大村 美里|        1|  女性|2007-11-25| 11| 156-0053|       東京都世田谷区桜**********|              S13035|        20150619|6-20091205-6|
|CS022103000002|  福山 はじめ|        9|  不明|2007-10-02| 11| 249-0006|     神奈川県逗子市逗子**********|              S14022|        20160909|0-00000000-0|
|CS002113000009|  柴田 真悠子|        1|  女性|2007-09-17| 11| 184-0014|  東京都小金井市貫井南町******...|              S13002|        20160304|0-00000000-0|
|CS004115000014|    松井 京子|        1|  女性|2007-08-09| 11| 165-0031|     東京都中野区上鷺宮**********|         

---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
# customer_idごとにランク付けする場合
# df_receipt.withColumn("rank", F.rank().over(Window.partitionBy("customer_id").orderBy(F.col("amount").desc()))).show()
df_receipt.withColumn("rank", F.rank().over(Window.orderBy(F.col("amount").desc())))\
          .select(["customer_id", "amount", "rank"]).show(10)

+--------------+------+----+
|   customer_id|amount|rank|
+--------------+------+----+
|CS011415000006| 10925|   1|
|ZZ000000000000|  6800|   2|
|CS028605000002|  5780|   3|
|CS015515000034|  5480|   4|
|ZZ000000000000|  5480|   4|
|ZZ000000000000|  5480|   4|
|ZZ000000000000|  5440|   7|
|CS021515000089|  5440|   7|
|ZZ000000000000|  5280|   9|
|CS017414000114|  5280|   9|
+--------------+------+----+
only showing top 10 rows



---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_receipt.withColumn("row_number", F.row_number().over(Window.orderBy(F.col("amount").desc())))\
          .select(["customer_id", "amount", "row_number"]).show(10)

+--------------+------+----------+
|   customer_id|amount|row_number|
+--------------+------+----------+
|CS011415000006| 10925|         1|
|ZZ000000000000|  6800|         2|
|CS028605000002|  5780|         3|
|CS015515000034|  5480|         4|
|ZZ000000000000|  5480|         5|
|ZZ000000000000|  5480|         6|
|ZZ000000000000|  5440|         7|
|CS021515000089|  5440|         8|
|ZZ000000000000|  5280|         9|
|CS017414000114|  5280|        10|
+--------------+------+----------+
only showing top 10 rows



---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
df_receipt.count()

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
df_receipt.select("customer_id").distinct().count()

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.groupBy("store_cd").sum("amount", "quantity")\
          .withColumnRenamed("sum(amount)", "amount")\
          .withColumnRenamed("sum(quantity)", "quantity")\
          .show()

+--------+------+--------+
|store_cd|amount|quantity|
+--------+------+--------+
|  S14024|736323|    2417|
|  S14045|458484|    1398|
|  S13020|796383|    2383|
|  S13038|708884|    2337|
|  S14047|338329|    1041|
|  S12014|725167|    2358|
|  S13037|693087|    2344|
|  S13051|107452|     354|
|  S12013|787513|    2425|
|  S14046|412646|    1354|
|  S14011|805724|    2434|
|  S13016|793773|    2432|
|  S13035|715869|    2219|
|  S14006|712839|    2284|
|  S14027|714550|    2303|
|  S13008|809288|    2491|
|  S14025|755581|    2394|
|  S13031|705968|    2336|
|  S14036|203694|     635|
|  S14040|701858|    2233|
+--------+------+--------+
only showing top 20 rows



---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupBy("customer_id").max("sales_ymd").show(10)

+--------------+--------------+
|   customer_id|max(sales_ymd)|
+--------------+--------------+
|CS024415000195|      20190907|
|CS011515000044|      20181226|
|CS017415000245|      20190615|
|CS011512000113|      20190829|
|CS025414000093|      20191016|
|CS010605000007|      20181009|
|CS011415000097|      20180909|
|CS018515000090|      20190717|
|CS015415000222|      20190928|
|CS038605000003|      20190910|
+--------------+--------------+
only showing top 10 rows



---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupBy("customer_id").min("sales_ymd").show(10)

+--------------+--------------+
|   customer_id|min(sales_ymd)|
+--------------+--------------+
|CS024415000195|      20170119|
|CS011515000044|      20170505|
|CS017415000245|      20180110|
|CS011512000113|      20181213|
|CS025414000093|      20170806|
|CS010605000007|      20170125|
|CS011415000097|      20170108|
|CS018515000090|      20170131|
|CS015415000222|      20170313|
|CS038605000003|      20170321|
+--------------+--------------+
only showing top 10 rows



---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
df_receipt_min = df_receipt.groupBy("customer_id")\
                            .agg({"sales_ymd":"min"})
df_receipt_max = df_receipt.groupBy("customer_id")\
                            .agg({"sales_ymd":"max"})
df_receipt_merge = df_receipt_min.join(df_receipt_max, df_receipt_min["customer_id"]==df_receipt_max["customer_id"])

In [ ]:
df_receipt_merge.filter(F.col("min(sales_ymd)") != F.col("max(sales_ymd)")).show()

+--------------+--------------+--------------+--------------+
|   customer_id|min(sales_ymd)|   customer_id|max(sales_ymd)|
+--------------+--------------+--------------+--------------+
|CS024415000195|      20170119|CS024415000195|      20190907|
|CS011515000044|      20170505|CS011515000044|      20181226|
|CS017415000245|      20180110|CS017415000245|      20190615|
|CS011512000113|      20181213|CS011512000113|      20190829|
|CS025414000093|      20170806|CS025414000093|      20191016|
|CS010605000007|      20170125|CS010605000007|      20181009|
|CS011415000097|      20170108|CS011415000097|      20180909|
|CS018515000090|      20170131|CS018515000090|      20190717|
|CS015415000222|      20170313|CS015415000222|      20190928|
|CS038605000003|      20170321|CS038605000003|      20190910|
|CS028515000242|      20190106|CS028515000242|      20190804|
|CS040215000044|      20170219|CS040215000044|      20190924|
|CS049513000008|      20181220|CS049513000008|      20190927|
|CS00241

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupBy("store_cd").avg("amount").orderBy(F.col("avg(amount)").desc()).show(5)

+--------+------------------+
|store_cd|       avg(amount)|
+--------+------------------+
|  S13052|402.86746987951807|
|  S13015|351.11196043165467|
|  S13003| 350.9155188246097|
|  S14010|348.79126213592235|
|  S13001| 348.4703862660944|
+--------+------------------+
only showing top 5 rows



---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
import numpy as np
import scipy as sp


@F.udf(DoubleType())
def udf_median(values_list):
    med = np.median(values_list)
    return float(med)

@F.udf(DoubleType())
def udf_mode(values_list):
    mode, count = sp.stats.mode(values_list)
    return float(count)

In [ ]:
df_receipt.groupBy("store_cd").agg(udf_median(F.collect_list('amount')).alias('median_amount')).orderBy(F.col("median_amount").desc()).show(5)

+--------+-------------+
|store_cd|median_amount|
+--------+-------------+
|  S13052|        190.0|
|  S14010|        188.0|
|  S14050|        185.0|
|  S14040|        180.0|
|  S13003|        180.0|
+--------+-------------+
only showing top 5 rows



---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [ ]:
# df_receipt.groupby(["store_cd", "product_cd"]).count().orderBy(F.col("count").desc()).show()
df_receipt.groupBy("store_cd").agg(udf_mode(F.collect_list('product_cd')).alias('mode_product_cd')).orderBy(F.col("mode_product_cd").desc()).show()

+--------+---------------+
|store_cd|mode_product_cd|
+--------+---------------+
|  S14027|          152.0|
|  S14012|          142.0|
|  S14028|          140.0|
|  S13031|          115.0|
|  S12030|          115.0|
|  S12013|          107.0|
|  S14024|           96.0|
|  S13044|           96.0|
|  S12029|           92.0|
|  S13037|           88.0|
|  S13004|           88.0|
|  S13032|           85.0|
|  S14040|           80.0|
|  S13020|           79.0|
|  S13002|           78.0|
|  S13008|           77.0|
|  S12007|           72.0|
|  S14046|           71.0|
|  S14022|           71.0|
|  S14034|           71.0|
+--------+---------------+
only showing top 20 rows



---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [ ]:
# df_receipt.select(F.stddev(F.col("amount")).alias("std")).show()
df_receipt.groupBy("store_cd").agg(F.variance(F.col("amount")).alias("variance")).orderBy(F.col("variance").desc()).show(5)

+--------+------------------+
|store_cd|          variance|
+--------+------------------+
|  S13052| 441863.2525262341|
|  S14011|306442.24243156874|
|  S14034| 297068.3927400612|
|  S13001| 295558.8426177121|
|  S13015| 295427.1970858535|
+--------+------------------+
only showing top 5 rows



---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [ ]:
# df_receipt.select(F.stddev(F.col("amount")).alias("std")).show()
df_receipt.groupBy("store_cd").agg(F.stddev(F.col("amount")).alias("std")).orderBy(F.col("std").desc()).show(5)

+--------+-----------------+
|store_cd|              std|
+--------+-----------------+
|  S13052|664.7279537722436|
|  S14011|553.5722558361905|
|  S14034| 545.039808399406|
|  S13001|543.6532374756101|
|  S13015|543.5321490821435|
+--------+-----------------+
only showing top 5 rows



---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
df_receipt.select("amount").summary().show()

+-------+------------------+
|summary|            amount|
+-------+------------------+
|  count|            104681|
|   mean| 320.5600825364679|
| stddev|477.70274931379606|
|    min|                10|
|    25%|               102|
|    50%|               170|
|    75%|               288|
|    max|             10925|
+-------+------------------+



---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df_receipt.groupBy("store_cd").agg(F.mean(F.col("amount")).alias("mean_amount")).filter(F.col("mean_amount")>=330).show()

+--------+------------------+
|store_cd|       mean_amount|
+--------+------------------+
|  S14045| 330.0820734341253|
|  S13020|  337.879932117098|
|  S14047| 330.0770731707317|
|  S12013|330.19412997903567|
|  S14011| 335.7183333333333|
|  S14010|348.79126213592235|
|  S13004|330.94394904458596|
|  S13052|402.86746987951807|
|  S13003| 350.9155188246097|
|  S13019|330.20861587554845|
|  S13001| 348.4703862660944|
|  S13015|351.11196043165467|
|  S14026|332.34058847239015|
+--------+------------------+



---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
df_receipt.filter(F.col("customer_id").rlike('^(?!Z).*'))\
          .groupBy("customer_id").agg(F.sum(F.col("amount")))\
          .agg(F.mean(F.col("sum(amount)"))).show()

+-----------------+
| avg(sum(amount))|
+-----------------+
|2547.742234529256|
+-----------------+



---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [ ]:
df_amount_sum = df_receipt.filter(F.col("customer_id").rlike('^(?!Z).*'))\
                        .groupBy("customer_id").agg(F.sum(F.col("amount")))
amount_mean = df_amount_sum.agg(F.mean(F.col("sum(amount)"))).first()[0]
df_amount_sum.filter(F.col("sum(amount)")>=amount_mean).show(10)

+--------------+-----------+
|   customer_id|sum(amount)|
+--------------+-----------+
|CS024415000195|       4638|
|CS025414000093|       3929|
|CS010605000007|       6420|
|CS011415000097|       3462|
|CS018515000090|       3756|
|CS015415000222|      11472|
|CS038605000003|       2833|
|CS049513000008|       4450|
|CS017514000012|       4699|
|CS020515000105|       5283|
+--------------+-----------+
only showing top 10 rows



---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [ ]:
df_receipt.show()

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|
| 20181205| 1543968000|  S14024|      1102|             2|CS024514000042|P080101005|       

In [ ]:
df_store.show()

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S12014|    千草台店|           12|    千葉県|        千葉県千葉市稲毛区千草台一丁目|   チバケンチバシイナゲクチグサダイイ...|043-123-4003|  140.118|35.63559|    1698.0|
|  S13002|    国分寺店|           13|    東京都|              東京都国分寺市本多二丁目|トウキョウトコクブンジシホンダニチョウメ|042-123-4008| 139.4802|35.70566|    1735.0|
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...

In [ ]:
df_receipt.join(df_store, "store_cd", "inner").select(df_receipt.columns+["store_name"]).show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+----------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|store_name|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+----------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|  葛が谷店|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|    成城店|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|  二ツ橋店|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|  新山下店|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|    大和店|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|    狛江店|
| 20181205| 1

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [ ]:
df_product.show()

+----------+-----------------+------------------+-----------------+----------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|
+----------+-----------------+------------------+-----------------+----------+---------+
|P040101001|                4|               401|            40101|       198|      149|
|P040101002|                4|               401|            40101|       218|      164|
|P040101003|                4|               401|            40101|       230|      173|
|P040101004|                4|               401|            40101|       248|      186|
|P040101005|                4|               401|            40101|       268|      201|
|P040101006|                4|               401|            40101|       298|      224|
|P040101007|                4|               401|            40101|       338|      254|
|P040101008|                4|               401|            40101|       420|      315|
|P040101009|         

In [ ]:
df_category.show()

+-----------------+-------------------+------------------+--------------------+-----------------+-------------------+
|category_major_cd|category_major_name|category_medium_cd|category_medium_name|category_small_cd|category_small_name|
+-----------------+-------------------+------------------+--------------------+-----------------+-------------------+
|                4|               惣菜|               401|              御飯類|            40101|             弁当類|
|                4|               惣菜|               401|              御飯類|            40102|             寿司類|
|                4|               惣菜|               402|              佃煮類|            40201|         魚介佃煮類|
|                4|               惣菜|               402|              佃煮類|            40202|         海草佃煮類|
|                4|               惣菜|               402|              佃煮類|            40203|         野菜佃煮類|
|                4|               惣菜|               402|              佃煮類|            40204|          

In [ ]:
df_product.join(df_category, ["category_major_cd", "category_medium_cd", "category_small_cd"], "inner").select(df_product.columns+["category_small_name"]).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+-------------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|category_small_name|
+----------+-----------------+------------------+-----------------+----------+---------+-------------------+
|P040101001|                4|               401|            40101|       198|      149|             弁当類|
|P040101002|                4|               401|            40101|       218|      164|             弁当類|
|P040101003|                4|               401|            40101|       230|      173|             弁当類|
|P040101004|                4|               401|            40101|       248|      186|             弁当類|
|P040101005|                4|               401|            40101|       268|      201|             弁当類|
|P040101006|                4|               401|            40101|       298|      224|             弁当類|
|P040101007|                4|       

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [ ]:
df_customer.show()

+--------------+-------------+---------+------+-------------------+---+---------+------------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender|          birth_day|age|postal_cd|                             address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+-------------------+---+---------+------------------------------------+--------------------+----------------+------------+
|CS021313000114|  大野 あや子|        1|  女性|1981-04-29 00:00:00| 37| 259-1113|      神奈川県伊勢原市粟窪**********|              S14021|        20150905|0-00000000-0|
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01 00:00:00| 66| 136-0076|          東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04 00:00:00| 42| 151-0053|        東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
|CS028811000001|  堀井 かおり|   

In [ ]:
df_receipt.show()

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|
| 20181205| 1543968000|  S14024|      1102|             2|CS024514000042|P080101005|       

In [ ]:
df_customer.join(df_receipt, "customer_id", "inner")\
            .filter((F.col("gender_cd")==1) & (F.col("customer_id").rlike('^(?!Z).*')))\
            .groupBy("customer_id").agg(F.sum(F.col("amount"))).show()

+--------------+-----------+
|   customer_id|sum(amount)|
+--------------+-----------+
|CS024415000195|       4638|
|CS011515000044|       2533|
|CS017415000245|       1289|
|CS011512000113|        686|
|CS025414000093|       3929|
|CS011415000097|       3462|
|CS018515000090|       3756|
|CS015415000222|      11472|
|CS028515000242|       1525|
|CS040215000044|       1439|
|CS049513000008|       4450|
|CS002411000018|       1054|
|CS017514000103|        348|
|CS010413000069|        278|
|CS038415000161|       2512|
|CS017514000012|       4699|
|CS030412000172|        766|
|CS002513000664|        398|
|CS002514000021|        413|
|CS034515000045|       1105|
+--------------+-----------+
only showing top 20 rows



---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [ ]:
df_receipt_tmp1.show()

+--------------+
|   customer_id|
+--------------+
|CS040214000008|
|CS010214000010|
|CS015415000185|
|CS028415000007|
|CS010214000002|
|CS007515000107|
|CS021514000045|
|CS016415000141|
|CS017415000097|
|CS031414000051|
|CS031414000073|
|CS021515000172|
|CS014415000077|
|CS032415000209|
|CS014214000023|
|CS039414000052|
|CS022515000226|
|CS021515000101|
|CS030214000008|
|CS021514000008|
+--------------+



In [ ]:
df_receipt_tmp1 = df_receipt.filter(F.col("customer_id").rlike('^(?!Z).*'))\
                            .groupBy("customer_id")\
                            .agg(F.count(F.col("sales_ymd")))\
                            .orderBy(F.col("count(sales_ymd)").desc()).select(["customer_id", "count(sales_ymd)"]).limit(20)
df_receipt_tmp2 = df_receipt.filter(F.col("customer_id").rlike('^(?!Z).*'))\
                            .groupBy("customer_id")\
                            .agg(F.sum(F.col("amount")))\
                            .orderBy(F.col("sum(amount)").desc()).select(["customer_id", "sum(amount)"]).limit(20)
df_receipt_tmp1.join(df_receipt_tmp2, "customer_id", "outer").show()

+--------------+----------------+-----------+
|   customer_id|count(sales_ymd)|sum(amount)|
+--------------+----------------+-----------+
|CS001605000009|            null|      18925|
|CS006515000023|            null|      18372|
|CS007514000094|            null|      15735|
|CS007515000107|              40|       null|
|CS009414000059|            null|      15492|
|CS010214000002|              42|       null|
|CS010214000010|              44|      18585|
|CS011414000106|            null|      18338|
|CS011415000006|            null|      16094|
|CS014214000023|              38|       null|
|CS014415000077|              38|       null|
|CS015415000185|              44|      20153|
|CS015515000034|            null|      15300|
|CS016415000101|            null|      16348|
|CS016415000141|              40|      18372|
|CS017415000097|              40|      23086|
|CS021514000008|              36|       null|
|CS021514000045|              40|       null|
|CS021515000089|            null| 

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [26]:
df_store.count() * df_product.count()

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [32]:
df_sales_amount_by_date = df_receipt.select(["sales_ymd", "amount"]).groupby("sales_ymd").agg(F.sum("amount"))
w = Window.orderBy("sales_ymd")
df_sales_amount_by_date = df_sales_amount_by_date.withColumn("lag_ymd", F.lag("sales_ymd").over(w))\
                                                 .withColumn("lag_amount", F.lag("sum(amount)").over(w))\
                                                 .withColumn("diff_amount", F.col("sum(amount)") - F.col("lag_amount"))

df_sales_amount_by_date.orderBy("sales_ymd").show(10)

+---------+-----------+----------+--------+-----------+
|sales_ymd|sum(amount)|lag_amount| lag_ymd|diff_amount|
+---------+-----------+----------+--------+-----------+
| 20170101|      33723|      null|    null|       null|
| 20170102|      24165|     33723|20170101|      -9558|
| 20170103|      27503|     24165|20170102|       3338|
| 20170104|      36165|     27503|20170103|       8662|
| 20170105|      37830|     36165|20170104|       1665|
| 20170106|      32387|     37830|20170105|      -5443|
| 20170107|      23415|     32387|20170106|      -8972|
| 20170108|      24737|     23415|20170107|       1322|
| 20170109|      26718|     24737|20170108|       1981|
| 20170110|      20143|     26718|20170109|      -6575|
+---------+-----------+----------+--------+-----------+
only showing top 10 rows



---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [34]:
df_sales_amount_by_date = df_receipt.select(["sales_ymd", "amount"]).groupby("sales_ymd").agg(F.sum("amount"))
w = Window.orderBy("sales_ymd")
df_sales_amount_by_date = df_sales_amount_by_date.withColumn("lag_ymd_d1", F.lag("sales_ymd").over(w))\
                                                 .withColumn("lag_amount_d1", F.lag("sum(amount)").over(w))\
                                                 .withColumn("lag_ymd_d2", F.lag("sales_ymd", offset=2).over(w))\
                                                 .withColumn("lag_amount_d2", F.lag("sum(amount)", offset=2).over(w))\
                                                 .withColumn("lag_ymd_d3", F.lag("sales_ymd", offset=3).over(w))\
                                                 .withColumn("lag_amount_d3", F.lag("sum(amount)", offset=3).over(w))
                                                 
df_sales_amount_by_date.dropna().orderBy("sales_ymd").show(10)                              

+---------+-----------+-------------+----------+-------------+----------+-------------+----------+
|sales_ymd|sum(amount)|lag_amount_d1|lag_ymd_d1|lag_amount_d2|lag_ymd_d2|lag_amount_d3|lag_ymd_d3|
+---------+-----------+-------------+----------+-------------+----------+-------------+----------+
| 20170104|      36165|        27503|  20170103|        24165|  20170102|        33723|  20170101|
| 20170105|      37830|        36165|  20170104|        27503|  20170103|        24165|  20170102|
| 20170106|      32387|        37830|  20170105|        36165|  20170104|        27503|  20170103|
| 20170107|      23415|        32387|  20170106|        37830|  20170105|        36165|  20170104|
| 20170108|      24737|        23415|  20170107|        32387|  20170106|        37830|  20170105|
| 20170109|      26718|        24737|  20170108|        23415|  20170107|        32387|  20170106|
| 20170110|      20143|        26718|  20170109|        24737|  20170108|        23415|  20170107|
| 20170111

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

# これで１００本終わりです。おつかれさまでした！